# VTA resnet18

In [2]:
import tvm
from tvm import relay
from dataclasses import dataclass
from pathlib import Path
from PIL import Image
from tqdm import tqdm
import numpy as np
from tvm_book.data.classification import ImageFolderDataset

def preprocess_image(
        image: np.ndarray,
        size: tuple[int] = (224, 224),
        mean: tuple[float] = (0.485, 0.456, 0.406),
        std: tuple[float] = (0.229, 0.224, 0.225)
    ):
    im = Image.fromarray(image)
    im = im.resize((256, 256), Image.Resampling.BILINEAR)
    ori_H, ori_W = im.size
    H, W = size
    space_W, space_H = (ori_W - W)//2, (ori_H - H)//2
    im = im.crop((space_W, space_H, ori_W-space_W, ori_H-space_H))
    image = np.array(im, dtype="float32")
    im.close()
    image = image/256
    image -= mean
    image /= std
    return image.astype(np.float32)

@dataclass
class ImageNet:
    root: str
    size: tuple[int] = (224, 224)
    mean: tuple[float] = (0.485, 0.456, 0.406)
    std: tuple[float] = (0.229, 0.224, 0.225)

    def __post_init__(self):
        self.root = Path(self.root) # 数据根目录
        self.valset = ImageFolderDataset(f"{self.root}/val")
        self.trainset = ImageFolderDataset(f"{self.root}/train")

    def calibrateset(self, calibrate_num: int = 200):
        """用于 TVM 量化的校准数据集
        """
        for k, (data, label) in tqdm(enumerate(self.trainset)):
            if k >= calibrate_num:
                break
            image = preprocess_image(data, self.size, self.mean, self.std)
            images = np.expand_dims(image, 0)
            images = images.transpose((0, 3, 1, 2))
            yield {"data": images}


def load_model(input_shape):
    from torchvision.models import resnet18, ResNet18_Weights
    import torch
    model = resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)
    data = torch.randn(*input_shape)
    model = torch.jit.trace(model.eval(), data)
    return model.eval()

input_shape = 1, 3, 224, 224
input_name = "data"
traced_model = load_model(input_shape)
mod, params = relay.frontend.from_pytorch(
    traced_model, 
    [(input_name, input_shape)], 
    # use_parser_friendly_name=True
)
dataset = ImageNet("/media/pc/data/lxw/home/data/datasets/ILSVRC/")
with tvm.transform.PassContext(opt_level=3):
    with relay.quantize.qconfig(
        skip_conv_layers=[],
        calibrate_mode="kl_divergence", 
        weight_scale="max",
        round_for_shift=True,
        # rounding="TONEAREST", # "UPWARD" or "TONEAREST"
        calibrate_skip_layers=[],
        skip_dense_layer=False,
    ):
        qmod = relay.quantize.quantize(mod, params, dataset.calibrateset(calibrate_num=200))

200it [01:27,  2.29it/s]


In [7]:
import vta
from tvm import rpc, autotvm

In [8]:
remote = rpc.LocalSession()
env = vta.get_env()
ctx = remote.ext_dev(0)
assert env.target.device_name == "vta"

In [ ]:
with autotvm.tophub.context(env.target):
    with tvm.transform.PassContext(opt_level=3):
        assert env.BLOCK_IN == env.BLOCK_OUT
        relay_prog = vta.top.graph_pack(
            qmod["main"],
            env.BATCH,
            env.BLOCK_OUT,
            env.WGT_WIDTH,
            start_name="nn.max_pool2d",
            stop_name="nn.adaptive_avg_pool2d",
            device_annot=(env.TARGET == "intelfocl"),
        )
    with vta.build_config(
            opt_level=3,
            disabled_pass={"AlterOpLayout",
                           "tir.CommonSubexprElimTIR"}
        ):
            lib = relay.build(relay_prog,
                              target=env.target,
                              params=params)
    # 将 inference library 发送到远程 RPC 服务器
    temp = tempdir()
    lib.export_library(temp.relpath("graphlib.tar"))
    remote.upload(temp.relpath("graphlib.tar"))
    lib = remote.load_module("graphlib.tar")

In [22]:
print(relay_prog)

fn (%data: Tensor[(1, 3, 224, 224), float32] /* ty=Tensor[(1, 3, 224, 224), float32] span=aten::_convolution_0.data:0:0 */) -> Tensor[(1, 1000), float32] {
  %0 = multiply(%data, 48.8058f /* ty=float32 */) /* ty=Tensor[(1, 3, 224, 224), float32] */;
  %1 = round(%0) /* ty=Tensor[(1, 3, 224, 224), float32] */;
  %2 = clip(%1, a_min=-127f, a_max=127f) /* ty=Tensor[(1, 3, 224, 224), float32] */;
  %3 = cast(%2, dtype="int8") /* ty=Tensor[(1, 3, 224, 224), int8] */;
  %4 = nn.conv2d(%3, meta[relay.Constant][0] /* ty=Tensor[(64, 3, 7, 7), int8] */, strides=[2, 2], padding=[3, 3, 3, 3], channels=64, kernel_size=[7, 7], out_dtype="int32") /* ty=Tensor[(1, 64, 112, 112), int32] */;
  %5 = add(%4, meta[relay.Constant][1] /* ty=Tensor[(64, 1, 1), int32] */) /* ty=Tensor[(1, 64, 112, 112), int32] */;
  %6 = nn.relu(%5) /* ty=Tensor[(1, 64, 112, 112), int32] */;
  %7 = cast(%6, dtype="int64") /* ty=Tensor[(1, 64, 112, 112), int64] */;
  %8 = fixed_point_multiply(%7, multiplier=1562234240, shift=-8

In [ ]:
# 下载 ImageNet categories
categ_url = "https://github.com/uwsampl/web-data/raw/main/vta/models"
categ_fn = "synset.txt"
download.download(f"{categ_url}/{categ_fn}", categ_fn)
synset = eval(open(categ_fn).read())
# 下载测试图片
image_url = "https://homes.cs.washington.edu/~moreau/media/vta/cat.jpg"
image_fn = "cat.png"
download.download(image_url, image_fn)
# 准备用于推理的测试图像
image = Image.open(image_fn).resize((224, 224))
plt.imshow(image)
plt.show()
image = np.array(image) - np.array([123.0, 117.0, 104.0])
image /= np.array([58.395, 57.12, 57.375])
image = image.transpose((2, 0, 1))
image = image[np.newaxis, :]
image = np.repeat(image, env.BATCH, axis=0)

# 生成图执行器（graph executor） `m`。
m = graph_executor.GraphModule(lib["default"](ctxes))
# 设置网络参数和输入
m.set_input(**params)
m.set_input("data", image)